# **Homework 2-1 Phoneme Classification**

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

### **Download data**

In [2]:
import os


# 要檢查的檔案路徑
filepath = "/content/data.zip"

# 檢查檔案是否存在
if os.path.isfile(filepath):
  print("檔案存在。")
  pass
else:
  print("檔案不存在。")
  !gdown --id '1seDU182JBfqxkGa503aSS-TRYRf0N5p3' --output data.zip
  !unzip data.zip
  !ls

檔案存在。


## release RAM

In [3]:

# torch.cuda.empty_cache
# !nvidia-smi
#!kill process_id
# import os, signal
# os.kill(os.getpid(), signal.SIGKILL)


## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [4]:

import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


# data size<br>
**Size of training data: (1229932, 429)<br>**
**Size of testing data: (451552, 429)**

## Create Dataset

In [5]:
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [6]:
from sklearn.model_selection import train_test_split

VAL_RATIO = 0.1 #0.2

train_x, val_x = train_test_split(train, random_state=31, train_size=(1-VAL_RATIO) )

train_y, val_y = train_test_split(train_label, random_state=31, train_size=(1-VAL_RATIO) )


print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))


# percent = int(train.shape[0] * (1 - VAL_RATIO))

# train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
# print('Size of training set: {}'.format(train_x.shape))
# print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (1106938, 429)
Size of validation set: (122994, 429)


# Size of training set: (983945, 429)</br>
# Size of validation set: (245987, 429)</br>
983945 = 5x47x53x79</br>
245987 = 7x35141

Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [7]:
import torch
BATCH_SIZE = 200 #64

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [8]:
import gc
del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

0

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [22]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()


        self.convolutionA = nn.Conv1d(in_channels=1, out_channels=39, kernel_size=10, stride=5, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
        self.flattenA = nn.Flatten(start_dim=1)

        self.convolutionB = nn.Conv1d(in_channels=20, out_channels=39, kernel_size=10, stride=5, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
        self.flattenB = nn.Flatten(start_dim=1)


        # self.convolutionC = nn.Conv1d(in_channels=39, out_channels=80, kernel_size=10, stride=5, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
        self.flattenC = nn.Flatten(start_dim=1)




        self.layer1 = nn.Linear(3276, 1024) #429, 1024  check input neuron number with print(x)

        self.layer2 = nn.Linear(1024, 512) #1024, 512


        self.layer3 = nn.Linear(512, 256) #512, 128


        self.layer4 = nn.Linear(256, 128) #512, 128


        self.out = nn.Linear(128, 39)  #128, 39

        self.batchnormA = nn.BatchNorm1d(39, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batchnormB = nn.BatchNorm1d(39, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

        self.batchnormC = nn.BatchNorm1d(585, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

        self.batchnorm1 = nn.BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batchnorm2 = nn.BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batchnorm3 = nn.BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batchnorm4 = nn.BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

        self.act_fnA = nn.ReLU()
        self.act_fnB = nn.ReLU()
        self.act_fnC = nn.ReLU()

        self.act_fn1 = nn.ReLU()
        self.act_fn2 = nn.ReLU()
        self.act_fn3 = nn.ReLU()
        self.act_fn4 = nn.ReLU()
        #self.act_fn = nn.Sigmoid()



        self.dtA = nn.Dropout(p=0.5)
        self.dtB = nn.Dropout(p=0.5)

        self.dtC = nn.Dropout(p=0.5)

        self.dt1 = nn.Dropout(p=0.5)
        self.dt2 = nn.Dropout(p=0.5)
        self.dt3 = nn.Dropout(p=0.5)
        self.dt4 = nn.Dropout(p=0.5)


    def forward(self, x):

        # print(x)
        # print(x.shape)
        

        x = x.reshape(x.shape[0], 1, 429)


        x = self.convolutionA(x)
        x = self.batchnormA(x)
        x = self.act_fnA(x)
        x = self.dtA(x)
        

        # x = self.convolutionB(x)
        # x = self.batchnormB(x)
        # x = self.act_fnB(x)
        # x = self.dtB(x)

        # x = self.convolutionC(x)
        # x = self.batchnormC(x)
        # x = self.act_fnC(x)
        # x = self.dtC(x)




        # print(x.shape)


        x = self.flattenA(x)

        # print(x.shape)



        x = self.layer1(x)
        x = self.batchnorm1(x)
        x = self.act_fn1(x)
        x = self.dt1(x)



        x = self.layer2(x)
        x = self.batchnorm2(x)
        x = self.act_fn2(x)
        x = self.dt2(x)



        x = self.layer3(x)
        x = self.batchnorm3(x)
        x = self.act_fn3(x)
        x = self.dt3(x)



        x = self.layer4(x)
        x = self.batchnorm4(x)
        x = self.act_fn4(x)
        x = self.dt4(x)

        x = self.out(x)
        
        return x

## Training

In [23]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [24]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

# *optimizer*

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
# unmount
# drive.flush_and_unmount()

In [25]:
# fix random seed for reproducibility
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 1000               # number of training epoch
learning_rate = 0.0001       # learning rate


# the path where checkpoint saved
model_path = './model.ckpt'
model_pathgd = '/content/drive/MyDrive/model.ckpt'
pred_path = './prediction.csv'
pred_pathgd = '/content/drive/MyDrive/prediction.csv'

# create model, define a loss function, and optimizer
model = Classifier().to(device)


# may load checkpoint
# model.load_state_dict(torch.load(model_path))


criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-05, amsgrad=False)

DEVICE: cuda


In [ ]:
# from google.colab import files
# files.download("model.ckpt")

In [ ]:
# start training

import pandas as pd



from google.colab import files

# pred in time
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)




best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    val_last = 0.0
    lastvalacc = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        # print(i)
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
        
            for i, data in enumerate(val_loader):
                val_last = val_acc
                # print(i)
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            # if (epoch % 10 == 0):
              
            #   torch.save(model.state_dict(), model_pathgd)
            #   print('saving drive done')
             


            lastvalacc = val_loss/len(val_loader)


            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))
            print( str(val_acc/len(val_set) - lastvalacc) )




            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                torch.save(model.state_dict(), model_pathgd)
                print('saving drive done')
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))





                # create model and load weights from checkpoint
                model2 = Classifier().to(device)
                model2.load_state_dict(torch.load(model_path))

                predict = []
                model2.eval() # set the model to evaluation mode
                with torch.no_grad():
                    for i, data in enumerate(test_loader):
                      inputs = data
                      inputs = inputs.to(device)
                      outputs = model2(inputs)
                      _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

                      for y in test_pred.cpu().numpy():
                        predict.append(y)


                with open('prediction.csv', 'w') as f:
                  f.write('Id,Class\n')
                  for i, y in enumerate(predict):
                    f.write('{},{}\n'.format(i, y))
                

                print('write pred to workspace done')




                  # predtodrive = pd.read_csv(pred_path)               
                  

                  # uploaded = drive.CreateFile({'title': 'prediction.csv'})
                  # uploaded.SetContentFile('prediction.csv')
                  # uploaded.Upload()
                  # torch.save(uploaded, pred_pathgd)
                  # print('saving pred to drive done'+ '\n')











            
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


[001/1000] Train Acc: 0.437546 Loss: 1.996455 | Val Acc: 0.582085 loss: 1.380612
-0.7985266101489455
saving drive done
saving model with acc 0.582
[002/1000] Train Acc: 0.532659 Loss: 1.595532 | Val Acc: 0.633023 loss: 1.209062
-0.5760388653043979
saving drive done
saving model with acc 0.633
[003/1000] Train Acc: 0.562592 Loss: 1.490784 | Val Acc: 0.650780 loss: 1.136345
-0.48556505514215464
saving drive done
saving model with acc 0.651
[004/1000] Train Acc: 0.580203 Loss: 1.428235 | Val Acc: 0.668073 loss: 1.080478
-0.41240479085588755
saving drive done
saving model with acc 0.668
[005/1000] Train Acc: 0.592093 Loss: 1.385055 | Val Acc: 0.676935 loss: 1.045828
-0.3688924457490883
saving drive done
saving model with acc 0.677
[006/1000] Train Acc: 0.601543 Loss: 1.352953 | Val Acc: 0.683586 loss: 1.018466
-0.33488006685269267
saving drive done
saving model with acc 0.684
[007/1000] Train Acc: 0.608487 Loss: 1.327785 | Val Acc: 0.690944 loss: 0.994208
-0.3032637992331926
saving drive d

TODO<br/>
loss vs time<br/>
acc displacement<br/>
model train reload

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [ ]:
# # create testing dataset
# test_set = TIMITDataset(test, None)
# test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# # create model and load weights from checkpoint
# model = Classifier().to(device)
# model.load_state_dict(torch.load(model_path))

Make prediction.

In [ ]:
# predict = []
# model.eval() # set the model to evaluation mode
# with torch.no_grad():
#     for i, data in enumerate(test_loader):
#         inputs = data
#         inputs = inputs.to(device)
#         outputs = model(inputs)
#         _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

#         for y in test_pred.cpu().numpy():
#             predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
# with open('prediction.csv', 'w') as f:
#     f.write('Id,Class\n')
#     for i, y in enumerate(predict):
#         f.write('{},{}\n'.format(i, y))

# Simple baseline
You should able to pass the simple baseline using the sample code provided.
# Strong baseline
Model architecture (layers? dimension? activation function?)<br> 
Training (batch size? optimizer? learning rate? epoch?)<br> 
Tips (batch norm? dropout? regularization?)